In [7]:
import pandas as pd

# Load the data
df = pd.read_csv('data_original/zillow_data.csv')

# Initial inspection
print(df.head())

# Separate the non-date and date columns
non_date_cols = df.columns[:5]
date_cols = df.columns[5:]

# Separate the data into non-date and date DataFrames
non_date_df = df[non_date_cols]
date_df = df[date_cols].apply(pd.to_numeric, errors='coerce')

# Print initial missing values stats
print("Initial missing values count:")
print(date_df.isnull().sum())

# Interpolate missing values for the date columns (which should be numeric)
date_df.interpolate(method='linear', axis=1, inplace=True)

# Check missing values after interpolation
print("\nMissing values count after interpolation:")
print(date_df.isnull().sum())

# Combine the non-date and interpolated date data back together
cleaned_df = pd.concat([non_date_df, date_df], axis=1)

# Melt the DataFrame to transform it into a long format
melted_df = cleaned_df.melt(id_vars=non_date_cols.tolist(), 
                            var_name='Date', 
                            value_name='Value')

# We should ensure the 'Date' column contains only date-like strings before converting them to datetime
melted_df = melted_df[melted_df['Date'].str.match(r'\d{4}-\d{2}-\d{2}')]

# Now, Convert 'Date' column to datetime type
melted_df['Date'] = pd.to_datetime(melted_df['Date'], format='%Y-%m-%d')

# Print the cleaned and melted DataFrame
print(melted_df.head())

# Print the info of the melted DataFrame
print(melted_df.info())


   RegionID  SizeRank       RegionName RegionType StateName     2000-01-31  \
0    102001         0    United States    country       NaN  121428.348338   
1    394913         1     New York, NY        msa        NY  216218.985144   
2    753899         2  Los Angeles, CA        msa        CA  222303.044856   
3    394463         3      Chicago, IL        msa        IL  152289.701354   
4    394514         4       Dallas, TX        msa        TX  125341.331449   

      2000-02-29     2000-03-31     2000-04-30     2000-05-31  ...  \
0  121641.979730  121906.914111  122475.146355  123129.113541  ...   
1  217137.793836  218065.112352  219944.218493  221890.097976  ...   
2  223130.294238  224232.182526  226424.570068  228822.354722  ...   
3  152430.677254  152699.168339  153367.107433  154170.557823  ...   
4  125397.158916  125461.338304  125628.005434  125847.751482  ...   

   1999-03-31  1999-04-30  1999-05-31  1999-06-30  1999-07-31  1999-08-31  \
0         NaN         NaN        

In [8]:
melted_df

,RegionID,SizeRank,RegionName,RegionType,StateName,Date,Value
0,102001,0,United States,country,NaN,2000-01-31,121428.348338
1,394913,1,"New York, NY",msa,NY,2000-01-31,216218.985144
2,753899,2,"Los Angeles, CA",msa,CA,2000-01-31,222303.044856
3,394463,3,"Chicago, IL",msa,IL,2000-01-31,152289.701354
4,394514,4,"Dallas, TX",msa,TX,2000-01-31,125341.331449
...,...,...,...,...,...,...,...
7911115,753929,935,"Zapata, TX",msa,TX,1999-12-31,117929.999579
7911116,394743,936,"Ketchikan, AK",msa,AK,1999-12-31,384967.024795
7911117,753874,937,"Craig, CO",msa,CO,1999-12-31,97704.295547
7911118,395188,938,"Vernon, TX",msa,TX,1999-12-31,89943.721726


In [9]:
melted_df.dropna(inplace=True)

In [10]:
melted_df

,RegionID,SizeRank,RegionName,RegionType,StateName,Date,Value
1,394913,1,"New York, NY",msa,NY,2000-01-31,216218.985144
2,753899,2,"Los Angeles, CA",msa,CA,2000-01-31,222303.044856
3,394463,3,"Chicago, IL",msa,IL,2000-01-31,152289.701354
4,394514,4,"Dallas, TX",msa,TX,2000-01-31,125341.331449
5,394692,5,"Houston, TX",msa,TX,2000-01-31,121045.187233
...,...,...,...,...,...,...,...
7911115,753929,935,"Zapata, TX",msa,TX,1999-12-31,117929.999579
7911116,394743,936,"Ketchikan, AK",msa,AK,1999-12-31,384967.024795
7911117,753874,937,"Craig, CO",msa,CO,1999-12-31,97704.295547
7911118,395188,938,"Vernon, TX",msa,TX,1999-12-31,89943.721726


In [11]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6213708 entries, 1 to 7911119
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   RegionID    int64         
 1   SizeRank    int64         
 2   RegionName  object        
 3   RegionType  object        
 4   StateName   object        
 5   Date        datetime64[ns]
 6   Value       float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 379.3+ MB


In [12]:
melted_df.to_csv('data_processing/final_zillow_data1.csv', index=False)

In [1]:
import hashlib

def calculate_md5_hash(file_path):
    md5_hash = hashlib.md5()
    with open(file_path, 'rb') as file:
        while chunk := file.read(4096):
            md5_hash.update(chunk)
    return md5_hash.hexdigest()

# Replace with the actual file path
file_path = 'data_processing/final_zillow_data1.csv'
md5 = calculate_md5_hash(file_path)
print(f"MD5 Hash: {md5}")


MD5 Hash: 7aeb305a6e566ff26216d3791b32df69
